<a href="https://colab.research.google.com/github/Blandalytics/baseball_snippets/blob/main/Auction_Calc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install packages

In [1]:
import pandas as pd
import numpy as np

# Definitions

User-defined league settings and value settings. Default settings are:
- Teams: 12
- Number of Hitters: 10
- Number of Pitchers: 8
- Number of Bench players: 5
- Number of Catchers: 1

- Per-Team Draft Budget: \$260
- Minimum Bid: \$1

- Hitter Split: 65%

In [2]:
# League Settings
TEAMS = 12
HITTERS = 10
PITCHERS = 8
BENCH = 5
CATCHERS = 1
MIN_BID = 1
BUDGET = 260
HITTER_SPLIT = 0.65

Replacement-level players are used to set our draftable subset of players. This will be smaller for leagues with fewer teams or players per team, and larger for leagues with more teams and more players per team.

This initial sample is defined as 110% of the number of drafted players, for both hitters and pitchers. Bench spots are evenly distributed between hitters and pitchers

In [3]:
# Derivative Definitions
PITCHER_SPLIT = 1 - HITTER_SPLIT

hitters_above_replacement = int(round(TEAMS * (HITTERS + BENCH/2) * 1.1,0))
pitchers_above_replacement = int(round(TEAMS * (PITCHERS + BENCH/2) * 1.1,0))
print(f'# of Replacement-Level Hitters : {hitters_above_replacement:.0f}')
print(f'# of Replacement-Level Pitchers : {pitchers_above_replacement:.0f}')

# of Replacement-Level Hitters : 165
# of Replacement-Level Pitchers : 139


Scoring categories are based on league settings. Default are `R`,`HR`,`RBI`,`SB`, and `BA` for hitters, and `W`,`K`,`SV`,`ERA`, and `WHIP` for pitchers.

Rate categories include: `BA`, `OBP`, `SLG`, and `OPS` for hitters, and `ERA`, `WHIP`, `K/9`, `BB/9`, `HR/9`, `K/BB`, `K%`, `BB%`, and `K-BB%` for pitchers. Volume categories are any not listed as rate categories.

For certain categories, a lower number is better. These are `K` and `CS` for hitters, and `BB`, `H`, `ER`, `BS`, `ERA`, `WHIP`, `L`, `HBP`, `HR`, `BB/9`, `HR/9` and `BB%` for pitchers.

In [4]:
# Define Scoring Categories - Hitting
volume_scoring_categories_h = ['R','HR','RBI','SB']
rate_scoring_categories_h = ['AVG'] # 'AVG' is the same thing as 'BA'
scoring_categories_h = volume_scoring_categories_h + rate_scoring_categories_h
inverted_categories_h = ['K','CS']

In [5]:
# Define Scoring Categories - Pitching
volume_scoring_categories_p = ['W','K','SV']
rate_scoring_categories_p = ['ERA','WHIP']
scoring_categories_p = volume_scoring_categories_p + rate_scoring_categories_p
inverted_categories_p = ['BB','H','ER','BS','ERA','WHIP','L','HBP','HR','BB/9','HR/9','BB%']

## Z-Score Functions

Volume-based stats are valued using a Z-Score calculation: (value - sample mean) / sample standard deviation

In [6]:
def volume_z_score(feat_col, population, sample):
    return (population[feat_col] - sample[feat_col].mean())/sample[feat_col].std()

For rate stats, this value is based on the impact of the player's performance AND the playing time they receive.

This is done by adding the total volume of the player's stat to a team of average players, and then dividing by the playing time of that same team of average players, plus the player

In [7]:
def rate_z_score(feat_col, playing_time_col, population, sample, n_players):
    population_playing_time = population[playing_time_col]
    sample_playing_time = sample[playing_time_col]
    league_playing_time = sample[playing_time_col].mean()

    population_volume = population[feat_col].mul(population_playing_time)
    sample_volume = sample[feat_col].mul(sample_playing_time)
    league_volume = sample[feat_col].mean() * league_playing_time

    sample_val = (sample_volume + (n_players-1) * league_volume) / (sample_playing_time + (n_players-1) * league_playing_time)
    population_val = (population_volume + (n_players-1) * league_volume) / (population_playing_time + (n_players-1) * league_playing_time)

    return (population_val - sample_val.mean())/sample_val.std()

# Hitters

In [8]:
# Load Projections
projections_hitters = pd.read_csv('https://docs.google.com/spreadsheets/d/1nnH9bABVxgD28KVj9Oa67bn9Kp5x2dD0nFiZ7jIfvmQ/export?gid=1029181665&format=csv')

# Select Replacement-level Sample, based on playing time ('PA')
sample_hitters  = projections_hitters.nlargest(hitters_above_replacement, 'PA')

In [9]:
# Calculate Z-Scores for Volume Stats
projections_hitters[[x+'_val' for x in volume_scoring_categories_h]] = volume_z_score(volume_scoring_categories_h,
                                                                                      projections_hitters,
                                                                                      sample_hitters)

# Z-Scores for Rate Stats
for x in rate_scoring_categories_h:
    projections_hitters[x+'_val'] = rate_z_score(x,'PA',
                                                 projections_hitters,
                                                 sample_hitters,
                                                 HITTERS)

# Invert relevant scoring categories
for x in list(set(scoring_categories_h) & set(inverted_categories_h)):
    projections_hitters[x+'_val'] = projections_hitters[x+'_val'].mul(-1)

# Total unadjusted Value: sum of all scoring category Z-Scores
projections_hitters['unadjusted_value'] = projections_hitters[[x+'_val' for x in scoring_categories_h]].sum(axis=1)

In [10]:
projections_hitters.loc[projections_hitters['Player']=='Shohei Ohtani',['Player','MLBAMID']+[x+'_val' for x in scoring_categories_h]+['unadjusted_value']].round(2)

,Player,MLBAMID,R_val,HR_val,RBI_val,SB_val,AVG_val,unadjusted_value
0,Shohei Ohtani,660271,3.48,3.44,2.33,3.11,2.45,14.8


Adjust the Unadjusted Values for the lowest replacement-level player to be 0, with the adjustment depending on if the player is a Catcher or not.

Catcher replacement player value is the (# of teams * # of catchers) best catcher. Subtract their Unadjusted Value from all other catchers' Unadjusted Values

Non-Catcher replacement player value is the (# of teams * (# of hitters - # of catchers)) best non-catcher

In [11]:
# Calculate Position Adjustment
projections_hitters['is_C'] = projections_hitters['Y! Pos'].fillna('UT').str.contains('C')
c_adj = projections_hitters.loc[projections_hitters['is_C'],'unadjusted_value'].nlargest(TEAMS * CATCHERS).min()
non_c_adj = projections_hitters.loc[~projections_hitters['is_C'],'unadjusted_value'].nlargest(int(TEAMS * (HITTERS - CATCHERS + BENCH/2))).min()
projections_hitters['ADJ'] = np.where(projections_hitters['is_C'],c_adj,non_c_adj)
print(f'Catcher Position Adjustment: {c_adj:.2f}')
print(f'Non-catcher Position Adjustment: {non_c_adj:.2f}')

# Total Adjusted Hitter Value
projections_hitters['adjusted_value'] = projections_hitters['unadjusted_value'].sub(projections_hitters['ADJ'])

Catcher Position Adjustment: -4.94
Non-catcher Position Adjustment: -3.21


In [12]:
projections_hitters.loc[projections_hitters['Player']=='William Contreras',['Player','MLBAMID','Y! Pos','unadjusted_value','ADJ','adjusted_value']].round(2)

,Player,MLBAMID,Y! Pos,unadjusted_value,ADJ,adjusted_value
38,William Contreras,661388,C,1.82,-4.94,6.76


In [13]:
projections_hitters.loc[projections_hitters['Player']=='Shohei Ohtani',['Player','MLBAMID','Y! Pos','unadjusted_value','ADJ','adjusted_value']].round(2)

,Player,MLBAMID,Y! Pos,unadjusted_value,ADJ,adjusted_value
0,Shohei Ohtani,660271,UT/SP,14.8,-3.21,18.01


# Pitchers

In [14]:
# Load Projections
projections_pitchers = pd.read_csv('https://docs.google.com/spreadsheets/d/1nnH9bABVxgD28KVj9Oa67bn9Kp5x2dD0nFiZ7jIfvmQ/export?gid=354379391&format=csv')

ip_thresh = min(50,projections_pitchers['IP'].nlargest(TEAMS * PITCHERS).min())

# Select Replacement-level Sample, based on playing time ('IP')
sample_pitchers  = projections_pitchers.loc[projections_pitchers['IP'] >= ip_thresh]

In [15]:
# Z-Scores for Volume Stats
projections_pitchers[[x+'_val' for x in volume_scoring_categories_p]] = volume_z_score(volume_scoring_categories_p,
                                                                                       projections_pitchers,
                                                                                       sample_pitchers)

# Z-Scores for Rate Stats
for x in rate_scoring_categories_p:
    projections_pitchers[x+'_val'] = rate_z_score(x,'IP',
                                                  projections_pitchers,
                                                  sample_pitchers,
                                                  PITCHERS)

# Invert relevant scoring categories
for x in list(set(scoring_categories_p) & set(inverted_categories_p)):
    projections_pitchers[x+'_val'] = projections_pitchers[x+'_val'].mul(-1)

# Total unadjusted Value
projections_pitchers['unadjusted_value'] = projections_pitchers[[x+'_val' for x in scoring_categories_p]].sum(axis=1)

# Position Adjustment for Catchers
projections_pitchers['ADJ'] = projections_pitchers['unadjusted_value'].nlargest(int(TEAMS * (PITCHERS + BENCH/2))).min()

# Total Adjusted Hitter Value
projections_pitchers['adjusted_value'] = projections_pitchers['unadjusted_value'].sub(projections_pitchers['ADJ'])

In [16]:
projections_pitchers.loc[projections_pitchers['Name']=='Tarik Skubal',['Name','MLBAMID','unadjusted_value','ADJ','adjusted_value']].round(2)

,Name,MLBAMID,unadjusted_value,ADJ,adjusted_value
0,Tarik Skubal,669373,12.22,1.15,11.06


# Combined

Pitchers and hitters are combined into a single dataframe

In [17]:
combined_value_df = (
    pd.concat(
        [
            projections_hitters[['Player','MLBAMID','Y! Pos','PA']+scoring_categories_h+['adjusted_value']],
            projections_pitchers.rename(columns={'Name':'Player'})[['Player','MLBAMID','IP']+scoring_categories_p+['adjusted_value']]
            ],
        ignore_index=True)
    [['Player','MLBAMID','Y! Pos','adjusted_value','PA']+scoring_categories_h+['IP']+scoring_categories_p]
)
combined_value_df['Y! Pos'] = combined_value_df['Y! Pos'].fillna('P')

Every player selected in an auction draft must be at least at the minimum bid value, so the dollar allocation needs to reflect this.

Calculate the allocation of Value budget available to hitters and pitchers, based on league settings.

In [18]:
# Dollar Conversion
non_replacement_dollars = (TEAMS * BUDGET) - (TEAMS * (HITTERS + PITCHERS + BENCH) * MIN_BID)
total_hitter_dollars = non_replacement_dollars * HITTER_SPLIT
total_pitcher_dollars = non_replacement_dollars * PITCHER_SPLIT
print(f'Value Budget Available: ${non_replacement_dollars:,.0f}')
print(f'Hitter Budget Available: ${total_hitter_dollars:,.2f}')
print(f'Pitcher Budget Available: ${total_pitcher_dollars:,.2f}')

Value Budget Available: $2,844
Hitter Budget Available: $1,848.60
Pitcher Budget Available: $995.40


In [19]:
total_hitter_value = projections_hitters.loc[projections_hitters['adjusted_value']>0,'adjusted_value'].sum()
total_pitcher_value = projections_pitchers.loc[projections_pitchers['adjusted_value']>0,'adjusted_value'].sum()
print(f'Total Hitter Value: {total_hitter_value:,.1f}')
print(f'Total Pitcher Value: {total_pitcher_value:,.1f}')

Total Hitter Value: 576.9
Total Pitcher Value: 429.8


In [20]:
hitter_dollars_per_value = total_hitter_dollars / total_hitter_value
pitcher_dollars_per_value = total_pitcher_dollars / total_pitcher_value
print(f'Hitter $ per Value: ${hitter_dollars_per_value:.2f}')
print(f'Pitcher $ per Value: ${pitcher_dollars_per_value:.2f}')

Hitter $ per Value: $3.20
Pitcher $ per Value: $2.32


In [21]:
# Apply the dollar conversions, based on whether they are a hitter or pitcher
combined_value_df['Auction $'] = MIN_BID + np.where(
    combined_value_df['Y! Pos']=='P',
    combined_value_df['adjusted_value'].mul(pitcher_dollars_per_value),
    combined_value_df['adjusted_value'].mul(hitter_dollars_per_value)
)

In [22]:
projected_auction_dollars = combined_value_df.loc[combined_value_df['Auction $']>0,'Auction $'].sum()
print(f'Projected Auction Dollars: ${projected_auction_dollars:,.2f}')
print(f'Defined Auction Dollars: ${TEAMS * BUDGET:,.2f}')

Projected Auction Dollars: $3,134.47
Defined Auction Dollars: $3,120.00


In [23]:
combined_value_df.loc[(combined_value_df['Player']=='Shohei Ohtani') & (combined_value_df['Y! Pos']!='P'),['Player','adjusted_value','Auction $']].round(2)

,Player,adjusted_value,Auction $
0,Shohei Ohtani,18.01,58.71


In [24]:
combined_value_df.loc[(combined_value_df['Player']=='Tarik Skubal'),['Player','adjusted_value','Auction $']].round(2)

,Player,adjusted_value,Auction $
619,Tarik Skubal,11.06,26.62


Apply a fudge factor to get the Projected dollar amounts to line up with the Defined dollar amounts

In [25]:
fudge_factor = (TEAMS * BUDGET) / projected_auction_dollars
combined_value_df['Auction $'] = combined_value_df['Auction $'].mul(fudge_factor)
print(f'Fudge Factor: {fudge_factor:.3f}')

projected_auction_dollars_adj = combined_value_df.loc[combined_value_df['Auction $']>0,'Auction $'].sum()
print(f'Projected Auction Dollars (adjusted): ${projected_auction_dollars_adj:,.2f}')
print(f'Defined Auction Dollars: ${TEAMS * BUDGET:,.2f}')

Fudge Factor: 0.995
Projected Auction Dollars (adjusted): $3,120.00
Defined Auction Dollars: $3,120.00


In [26]:
combined_value_df.loc[(combined_value_df['Player']=='Shohei Ohtani') & (combined_value_df['Y! Pos']!='P'),['Player','adjusted_value','Auction $']].round(2)

,Player,adjusted_value,Auction $
0,Shohei Ohtani,18.01,58.44


In [27]:
combined_value_df.loc[(combined_value_df['Player']=='Tarik Skubal'),['Player','adjusted_value','Auction $']].round(2)

,Player,adjusted_value,Auction $
619,Tarik Skubal,11.06,26.5
